# Data Preparation

Dataset: https://www.kaggle.com/datasets/nasa/astronaut-yearbook/data

<a href="https://colab.research.google.com/github/simecek/dspracticum2024/tree/main/lesson09/astronauts_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### Download the dataset:

In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = 'evaklimentov'
os.environ['KAGGLE_KEY'] = '2a5363f316b3cce3414b917f4f888067'

!kaggle datasets download -d nasa/astronaut-yearbook -p {'.'}

In [3]:
import zipfile

dataset_path = './'

with zipfile.ZipFile('astronaut-yearbook.zip', 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

In [ ]:
import pandas as pd

csv_filepath = 'astronauts.csv'
df = pd.read_csv(csv_filepath)
df

## 0. Data Exploration

- get insights to your dataset (you can laso look at the original dataset description at https://www.kaggle.com/datasets/nasa/astronaut-yearbook/data)


In [ ]:
# TODO: get insight into your dataset, use functions as
# df.describe(), df.info()
# and answer the next questions:

What is the Gender ratio?

In [ ]:
# TODO: count distinct values of values in the 'Gender' column and display the result (e.g. as a pie chart)

Plot histogram of values of one feature (1 column):

In [ ]:
import seaborn as sns

# TODO
# hint: use sns.histplot()

Was somebody on more than one `Mission`s?

(hint: Missions would be separated by a comma.)

In [ ]:
# TODO: parse the information in 'Mission' column and count the number of missions, then filter based on that number

## 1. Data Cleaning

**Look for duplicites:**

In [ ]:
# hint: use df.duplicated()

We can also look for duplicites only w.r.t. to one feature and not the entire sample (row):

In [ ]:
df.loc[df.duplicated(['Birth Date'], keep=False)]

In [ ]:
# TODO: are there any duplicates we should be concerned about?

**Check for inconsistencies:**

In [ ]:
# TODO are there any relations between the columns that should hold?

**Missing data:**

In [ ]:
df.info()

We can see that we have sone Null values in columns `Year`, `Group`, `Alma Mater`, `Undergraduate Major`, `Graduate Major`, `Undergraduate Major`, `Graduate Major`, `Military Rank`, `Military Branch`, `Missions`, `Death Date`, and `Death Mission`.



In [ ]:
# TODO: check few samples from our dataset with the missing value - how are we going to handle it?
df[pd.isna(df['Group'])]

Leave the missing value as it is for these features:

*(here the fact that the value is missing indicates an important sample property)*

In [ ]:
df['Group'] = df['Group'].fillna('None')

In [ ]:
# TODO: what are the other features where a missing value indicates some sample property?

Fill in with mean:

In [ ]:
df['Year'] = df['Year'].fillna(int(df['Year'].mean(numeric_only=True)))
df

Remove rows with missing value(s):

In [ ]:
df = df.dropna(subset=['Alma Mater'], how='all')
df

## 2. Feature manipulation:

In [27]:
df_transformed = df.copy()


#### Categorical features encoding:

Example - encoding the `Status` feature with One Hot Encoding:

In [ ]:
df_transformed['Status'].value_counts()

In [ ]:
ohe_status = pd.get_dummies(df_transformed[['Status']])
df_transformed = df_transformed.join(ohe_status)
df_transformed

What else can we handle the same way?

In [ ]:
# TODO

What about features with more distinct values?

In [ ]:
df_transformed['Graduate Major'].value_counts()

In [32]:
# TODO: look at other categorical features and come up with a way of encoding their values

Binning of `Graduate Major`s based on simple key words:

In [ ]:
def get_superior_category(major_str):
    if pd.isna(major_str):
        return 'None'
    if 'engineer' in major_str.lower():
        return 'Engineering'
    elif 'bio' in major_str.lower() or 'science' in major_str.lower() or 'astro' in major_str.lower() or 'physics' in major_str.lower() or 'math' in major_str.lower() or 'inf' in major_str.lower():
        return 'Science'
    elif 'med' in major_str.lower() or 'doctor' in major_str.lower() or 'health' in major_str.lower():
        return 'Medicine'
    elif 'business' in major_str.lower() or 'management' in major_str.lower():
        return 'Business'
    return 'Different'

df_transformed['Graduate Major'] = df_transformed.apply(lambda row: get_superior_category(row['Graduate Major']), axis=1)
df_transformed['Graduate Major'].value_counts()

In [ ]:
ohe_grad_df = pd.get_dummies(df_transformed[['Graduate Major']])
df_transformed = df_transformed.join(ohe_grad_df)
df_transformed

## 4. Dimensionality reduction

#### PCA:

In [ ]:
# boolean variables to int
df_transformed = df_transformed.apply(lambda col: col.astype(int) if col.dtypes == 'bool' else col)
# select only numerical features
df_numerical = df_transformed.select_dtypes(include='number')
df_numerical

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
components = pca.fit_transform(df_numerical)

# join the PCA info with our original df:
components = pd.DataFrame(components)
df_transformed['PCA 0'] = components[0]
df_transformed['PCA 1'] = components[1]

In [ ]:
import plotly.express as px

def show_reduced_dimensions_interactive(color_column, x='PCA 0', y='PCA 1'):
  fig = px.scatter(df_transformed,
                  x=x,
                  y=y,
                  color=color_column,
                  hover_name='Name',
                  hover_data=['Year', 'Group', 'Status', 'Birth Date',
                              'Birth Place', 'Gender', 'Alma Mater',
                              'Undergraduate Major', 'Graduate Major',
                              'Military Rank', 'Military Branch',
                              'Space Flights', 'Space Flight (hr)',
                              'Space Walks', 'Space Walks (hr)', 'Missions',
                              'Death Date', 'Death Mission']
                  )
  fig.show()

We can color the PCA based on different features: 

In [ ]:
show_reduced_dimensions_interactive('Gender')

In [ ]:
show_reduced_dimensions_interactive('Military Rank')